# Projeto: Criação de um código que retorne o Ke de uma ou mais ações usando o CAPM.

In [1]:
#importando bibliotecas
import time
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import pandas_datareader.data as web
from selenium import webdriver

## Puxando dados ações

In [12]:
print("""Colocar as ações no mesmo formato que estão no yahoo finance e uma por uma.
Exemplo:
input_1 = ITSA4.SA
input_2 = VIVT3.SA
""")

# Ações
n = int(input('Quantas ações deseja analizar? '))
tickers = []
for i in range(n):
    ticker = input('Quais as ações que deseja analizar? ').upper()
    tickers.append(ticker)

# Indice
index = []
index = input('Qual indice usara como referencia? ').upper()
tickers_2 = tickers.copy()
tickers_2.append(index)

# Data de inicio
data = input('Qual a data de início para o cálculo do beta? (aaaa-mm-dd)')
yf.pdr_override()
df = web.get_data_yahoo(tickers_2, start = data, end = dt.date.today())

# Preço de fechamento ajustado
df_adj = df['Adj Close']

Colocar as ações no mesmo formato que estão no yahoo finance e uma por uma.
Exemplo:
input_1 = ITSA4.SA
input_2 = VIVT3.SA

Quantas ações deseja analizar? 3
Quais as ações que deseja analizar? itsa4.sa
Quais as ações que deseja analizar? vale3.sa
Quais as ações que deseja analizar? vivt3.sa
Qual indice usara como referencia? ^bvsp
Qual a data de início para o cálculo do beta? (aaaa-mm-dd)2017-01-01
[*********************100%***********************]  4 of 4 completed


### Retorno

In [13]:
# Retorno diario
def daily_return(df):
    df_daily_return = df.copy()
    for i in df.columns[:]:
        for j in range(1, len(df)):
            df_daily_return[i][j] = ((df[i][j] - df[i][j-1])/df[i][j-1]) * 100
        df_daily_return[i][0] = 0
    return df_daily_return
daily_r_d = daily_return(df_adj)
daily_r_d = daily_r_d[1:].dropna()

### Taxa livre de risco

In [8]:
# Pegando a taxa livre de risco do site do tesouro
url = 'https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm#0'
navegador = webdriver.Chrome()
navegador.get(url)
time.sleep(5)
rf = navegador.find_element_by_xpath('//*[@id="td-precos_taxas-tab_1"]/div/div[9]/table/tbody[3]/tr/td[2]/span').text
ano = navegador.find_element_by_xpath('//*[@id="td-precos_taxas-tab_1"]/div/div[9]/table/tbody[3]/tr/td[1]/span/span/span').text
navegador.close()
rf = rf.replace(',','.')
print(f'A taxa livre de risco usada é o tesouro pré fixado {ano} com retorno de {rf} ao ano.')
rf = float(rf.replace('%',''))

A taxa livre de risco usada é o tesouro pré fixado 2031 com retorno de 10.92% ao ano.


### Premio de risco

In [18]:
# Pegando o premio de mercado no site do Damodaran
dam = pd.read_html('https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html', skiprows = 0, header = 0)[0]
dam = pd.DataFrame(dam)
dam = dam.set_index('Country')
rp_d = dam.loc['Brazil','Total Equity Risk Premium']
rp_d = float(rp_d.replace('%',''))

# Adotando o retorno médio do indice no periodo escolhido
rm = daily_r_d[index].mean() * 252
rp_a = round((rm - rf), 2)

print(f'''O premio de risco de mercado para o brasil segundo o site do Damodaran é {rp_d}% ao ano.
O premio de risco de mercado dado o indice escolhido e a data escolhida é {rp_a}% ao ano.''')

O premio de risco de mercado para o brasil segundo o site do Damodaran é 7.63% ao ano.
O premio de risco de mercado dado o indice escolhido e a data escolhida é 6.59% ao ano.


## Calculando o Ke

In [15]:
if n == 1:
    beta = np.cov(daily_r_d[index], daily_r_d[tickers])[0][1] / np.var(daily_r_d[tickers])
    Ke = round(rf + beta * rp_a, 2)
    print(f'O retorno esperado é {ri}%.')
else:
    Ke = {}
    for i in tickers:
        beta = np.cov(daily_r_d[index], daily_r_d[i])[0][1] / np.var(daily_r_d[i])
        Ke[i] = round(rf + beta * rp_a, 2)
        print (f'O retorno esperado para {i} é de {Ke[i]}%.')
# Assumindo pesos iguais para as ações em um portifolio
pesos = 1/n * np.ones(n)
Ke_portfolio = round(sum(list(Ke.values()) * pesos), 2)
print (f'O retorno esperado para o portfólio é de {Ke_portfolio}%.')

O retorno esperado para ITSA4.SA é de 15.62%.
O retorno esperado para VALE3.SA é de 13.62%.
O retorno esperado para VIVT3.SA é de 13.81%.
O retorno esperado para o portfólio é de 14.35%.
